# Introdução

## A fonte de dados

Escolhi a fonte de dados abertos da CVM contendo a cotação diária dos fundos de investimentos negociados no mercado brasileiro. Essa fonte é atualizada diariamente com os dados de fechamento do dia anterior e as contações são agrupadas por arquivos correspondentes a cada mês do ano.

Esse é um cenário bem comum em fontes de dados abertas, uma série de arquivos no formato CSV agrupapando informações de acordo com a data, então  a solução que vamos implementar durante o tutorial é reaproveitável para outras fontes de dados.

Primeiro é importante analisar a fonte de dados, entender como ela está estruturada, quais campos compõem o dataset e como nós podemos automatizar a coleta dos dados.

A fonte de dados contendo a cotação diária dos fundos pode ser acessada através do [portal de dados abertos](http://www.dados.gov.br) pelo link:

[http://www.dados.gov.br/dataset/fi-doc-inf_diario](http://www.dados.gov.br/dataset/fi-doc-inf_diario)

Aqui tem uma descrição rápida sobre as informações que existem no dataset:
![](img/descricao_dataset.png)

Além de um link para o recurso contendo o dicionário de dados que descreve cada campo do dataset, é **sempre muito importante analisar o dicionário de dados do dataset, se disponível.** Não vou entrar em detalhes sobre as descrições do dicionário para não estender o tutorial, mas sugiro que você olhe para ser familiarizar com esse tipo de formato.

![](img/link_dicionario.png)

## Como automatizar o download 

Agora partindo para como nós podemos automatizar a extração dos datasets, como podemos observar no site existe um link para todos os datasets dos últimos 12 meses. Primeiro precisamos analisar se existe um padrão nas urls de download dos arquivos, para isso vamos copiar alguns endereços e compará-los.

Para copiar os endereços clique com o botão direito no botão **"Ir para recurso"** e selecione a opção **"Copiar link"**:

![](img/link_recurso.png)

Copiei três links e vamos compará-los a seguir:

[http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201907.csv](http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201907.csv)

[http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201910.csv](http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_201910.csv)

[http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202003.csv](http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202003.csv)

Como podemos observar existe um padrão claro nos links:`dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_YYYYMM.csv`. A única variação é o mês e o ano de referência do dataset. Com essa análise nossa tarefa de automatizar o download dos datasets ficou mais simples, porque agora podemos gerar sistematicamente o intervalo de datas que queremos baixar.

# Let's CODE

## Extração

O processo de extração consiste nesse caso em fazer o download de todos os arquivos da janela de tempo q nos interessa. Mas precisamos visualizar todas as etapas que precisam ser completadas até que seja possível fazer o download desses arquivos:

1. Automatizar a geração do nome dos arquivos: já que a única coisa que varia nos links é o nome dos arquivos precisamos automatizar a geração desses nomes de acordo com a janela de tempo do nosso interesse;

2. Requisitar o arquivo: precisamos enviar uma requisição para o portal de dados abertos do arquivo que queremos fazer o download;

3. Salvar arquivo: o portal de dados abertos vai nos enviar o arquivo requisitado e precisamos salvá-lo no nosso computador.

### Gerando datas

In [1]:
from datetime import datetime, timedelta

Primeiramente vamos criar uma função para facilitar a criação da lista de datas que nós queremos baixar do site:

In [49]:
def generate_dates(initial_date, final_date, increment, format):
    """
    Função para gerar uma lista de datas no formato string.
    
    Parâmetros:
        initial_date = Data inicial da contagem;
        
        final_date = Data final da contagem;
        
        increment = Quanto deve ser incrementado da data, em dias, para cada iteração;
        
        format = Formato da data de saída obedecendo os códigos disponíveis na documentação do python:
                 https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes
    """
    date_list = []
    
    while initial_date < final_date:
        date_list.append(initial_date.strftime(format))
        initial_date = initial_date + timedelta(days=increment)
        
    return date_list

### Fazendo a requisição e salvando os arquivos 

Vamos trabalhar agora com a biblioteca python [requests](https://requests.readthedocs.io/en/master/) que facilita a implementação de requisições HTTP, que é essencialmente o tipo de requisição que precisamos fazer para baixar os arquivos da nossa fonte de dados. 

Para fazer a requisição utilizaremos o méodo `requests.get()` que recebe como primeiro parâmetro a URL que queremos requisitar, podemos então checar se ocorreu algum erro através do [_status code_](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status) da resposta, o _status code_ `200` significa que a requisição foi bem sucedida. No fim, o conteúdo da resposta pode ser acessado através do atributo `.content`. Recomendo explorar a excelente [documentação da biblioteca](https://requests.readthedocs.io/en/latest/) caso queira conhecer melhor.

```python
>>> import requests

>>> response = requests.get(url)

>>> response.content
b"Conteúdo da página"
```

In [50]:
import requests

In [83]:
def download_files(dates, file_base_name='inf_diario_fi', dir='data/'):
    """
    Função para fazer o download de todos os arquivos das datas disponibilizadas.
    
    Parâmetros:
        dates = Lista de datas;
        
        file_base_name = Nome base para os arquivos que serão salvos;
        
        dir = Diretório onde os arquivo serão salvos.
    """
    saved_files = []
    # URL base para baixar os arquivos
    base_url = "http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS"
    
    for date in dates:
        # Contrói a URL de download do arquivo a partir da data
        url = "{}/inf_diario_fi_{}.csv".format(base_url, date)
        
        # Realiza a requisição
        response = requests.get(url)
                
        if response.status_code != 200:
            print("Erro ao requisitar o arquivo {}".format(url))
            
            # Caso ocorra um erro durante a requisição pular para a próxima data
            continue
        
        
        file_name = "{}{}_{}.csv".format(dir, file_base_name, date)
        
        # Salva o arquivo
        with open(file_name, 'wb') as f:
            f.write(response.content)

        saved_files.append(file_name)
        print('Arquivo {} salvo.'.format(file_name))
    
    return saved_files

Estamos prontos agora para fazer o download dos arquivos, vamos definir duas variáveis para a data inicial e final do período que queremos extrair:

In [84]:
initial_date = datetime(year=2019, month=5, day=1)
final_date = datetime(year=2020, month=4, day=30)

Com a função `generate_dates` vamos gerar a lista de datas do período definido. Precisamos passar a data inicial e final, o quanto queremos incrementar em dias para cada iteração, nesse caso como nossa fonte de dados tem um arquivo por mês devemos incrementar 31 dias a cada iteração. Como último parametrô a função recebe o formato da data de saída, devemos utilizar os [códigos de formatação descritos na documentação do Python](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes) para construir a data no formato que queremos, na fonte de dados a data dos arquivos está no formato: YYYYMM.

In [85]:
date_list = generate_dates(initial_date, final_date, 31, "%Y%m")

In [86]:
print(date_list)

['201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004']


Vamos execuar a função de download dos arquivos e assistir a magia acontecer:

In [87]:
file_list = download_files(date_list)

Arquivo data/inf_diario_fi_201905.csv salvo.
Arquivo data/inf_diario_fi_201906.csv salvo.
Arquivo data/inf_diario_fi_201907.csv salvo.
Arquivo data/inf_diario_fi_201908.csv salvo.
Arquivo data/inf_diario_fi_201909.csv salvo.
Arquivo data/inf_diario_fi_201910.csv salvo.
Arquivo data/inf_diario_fi_201911.csv salvo.
Arquivo data/inf_diario_fi_201912.csv salvo.
Arquivo data/inf_diario_fi_202001.csv salvo.
Arquivo data/inf_diario_fi_202002.csv salvo.
Arquivo data/inf_diario_fi_202003.csv salvo.
Arquivo data/inf_diario_fi_202004.csv salvo.


## Tratamento


Antes de entrar na etapa de tratamente é importante que você explore os dados para assim conseguir identificar exatamente quais transformações você deseja aplicar, se é necessário tratar algum campo ou filtrar alguma informação. Não vou demonstrar a etapa de análise pois é necessário um tutorial dedicado somente a esclarecer essa fase. Vamos partir do pressuposto que é necessário realizar as seguintes atividades:

1. Juntar todos os arquivos em um só;
2. Remover os caracteres especiais do CNPJ;
3. Manter somente as colunas: `CNPJ_FUNDO`, `VL_QUOTA`, `DT_COMPTC`, `VL_PATRIM_LIQ` e `NR_COTST`.

Daqui para frente vamos utilizar principalmente a biblioteca [Pandas](https://pandas.pydata.org/), que oferece uma série de ferramentas para facilitar a manipulação e análise de dados. O Pandas oferece uma estrutura de dados tabular chamada [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) que permite carregar dados de diferentes formatos.

In [97]:
import pandas as pd

### Juntando todos arquivos em um

Vamos utilizar a função [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) do pandas para ler os arquivos baixados e a função [concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html?highlight=concat#pandas-concat) para juntá-los em um só [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). É importante observar que o separador dos arquivos `csv` baixados não é a vírgula(`,`) e sim o ponto e vírgula(`;`), é necessário especificar essa informação ao ler o arquivo usando a função [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). O jeito mais rápido de descobrir qual é o separador de um arquivo `csv` é abrindo-o como um arquivo de texto através de um editor de texto, é possível observar qual o separador utilizado no conteúdo do arquivo.

Vamos então criar uma função para concatenar todos os arquivos lidos em um só [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html):

In [102]:
def concat_csvs(files, sep=','):
    """
    Concatena todos os arquivos em um só DataFrame
    
    Parâmetros:
        files = Lista do nome e caminho dos arquivos a serem concatenados;
        sep = Separador dos arquivos csv's.
    """
    # Gera uma lista onde cada DataFrame é um item da lista
    df_list = [pd.read_csv(file, sep=sep) for file in files]
    
    # Concatena todos os itens da lista em um só DataFrame
    df = pd.concat(df_list)
    
    return df

Vamos aproveitar da capacidade do Jupyter Notebook de executar [comandos shell](https://www.geeksforgeeks.org/basic-shell-commands-in-linux/) para listar todos os arquivos salvos no diretório `data/` e armazenar o resultado em uma variável:

In [94]:
files = !ls -d data/*

In [95]:
print(files)

['data/inf_diario_fi_201905.csv', 'data/inf_diario_fi_201906.csv', 'data/inf_diario_fi_201907.csv', 'data/inf_diario_fi_201908.csv', 'data/inf_diario_fi_201909.csv', 'data/inf_diario_fi_201910.csv', 'data/inf_diario_fi_201911.csv', 'data/inf_diario_fi_201912.csv', 'data/inf_diario_fi_202001.csv', 'data/inf_diario_fi_202002.csv', 'data/inf_diario_fi_202003.csv', 'data/inf_diario_fi_202004.csv']


Agora podemos executar a função `concat_csvs` para gerar nosso [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) resultante:

In [98]:
df = concat_csvs(files, sep=';')

In [101]:
print("Quantidade de linhas do DataFrame: {}".format(df.shape[0]))
print("Quantidade de colunas do DataFrame: {}".format(df.shape[1]))

Quantidade de linhas do DataFrame: 4105029
Quantidade de colunas do DataFrame: 8


In [104]:
print("Colunas do DataFrame: {}".format(df.columns.values))

Colunas do DataFrame: ['CNPJ_FUNDO' 'DT_COMPTC' 'VL_TOTAL' 'VL_QUOTA' 'VL_PATRIM_LIQ'
 'CAPTC_DIA' 'RESG_DIA' 'NR_COTST']


### Removendo caracteres especiais do CNPJ